# Importing Libraries

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints Mapping

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Dataset

In [5]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('C:\\Users\\mahat\\SignLanguageTranslation\\MP_DATA') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])
# actions = np.array(['hello', 'father', 'me', 'mother', 'yes', 'no', 'help', 'please', 'what', 'cat', 'repeate', 'eat', 'milk', 'more', 'fine', 'thanks', 'you', 'how', 'so-so'])


# Thirty videos worth of data for each image
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collecting Data

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        # Looping through actions
        for action in actions:
            # Looping through sequence:
            for sequence in range(no_sequences):
                #Looping throught video length
                for frame_num in range(sequence_length):
                    
                    # Read feed
                    ret, frame = cap.read()
                    # frame = cv2.flip(frame, 1) dont flip 
            
                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
                    print(results)
                    
                    # Draw landmarks
                    draw_landmarks(image, results)

                    
                    # Apply wait logic
                    if frame_num == 0: 
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA) #line width = 4
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(500)
                    else: 
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)

                    #Keypoint extraction
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints) #This saves the keypoints array into the file named as frame_num.npy
            
                   

                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
        cap.release()
        cv2.destroyAllWindows()

# One hot Encoding 

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label: num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(90, 30, 1662)

In [17]:
np.array(labels).shape

(90,)

In [19]:
X = np.array(sequences)

In [21]:
X.shape

(90, 30, 1662)

In [23]:
y = to_categorical(labels).astype(int)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [27]:
y_test.shape

(5, 3)

# Building the LSTM Model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [31]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [35]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=False, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [37]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [41]:
model.fit(X_train, y_train, validation_data= (X_test, y_test) , epochs=20)

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - categorical_accuracy: 0.9765 - loss: 0.3027 - val_categorical_accuracy: 1.0000 - val_loss: 0.2754
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - categorical_accuracy: 0.9511 - loss: 0.2850 - val_categorical_accuracy: 1.0000 - val_loss: 0.1844
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - categorical_accuracy: 0.9706 - loss: 0.2083 - val_categorical_accuracy: 1.0000 - val_loss: 0.1511
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.9608 - loss: 0.1436 - val_categorical_accuracy: 1.0000 - val_loss: 0.0614
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - categorical_accuracy: 0.9472 - loss: 0.1843 - val_categorical_accuracy: 1.0000 - val_loss: 0.0758
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - categorical_accuracy: 0.9628 - loss: 0.1256 - val_categorical_accuracy: 1.0000 - val_loss: 0.0472
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - categorical_accuracy: 0.9745 - loss: 0.0922 - val_categ

In [43]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 30, 64)              │         442,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,654,091 (6.31 MB)

 Trainable params: 551,363 (2.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,102,728 (4.21 MB)

In [45]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step


In [47]:
actions[np.argmax(res[4])]

'thanks'

In [49]:
actions[np.argmax(y_test[4])]

'thanks'

# Saving the model

In [73]:
import joblib

In [75]:
joblib.dump(model, 'lstm_final_model.keras')

['lstm_final_model.keras']

In [77]:
model = joblib.load('lstm_final_model.keras')

In [51]:
model.save('lstm_model.keras') # Dont run

# Evaluating Accuracy using Confusion Matrix

In [53]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [55]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [57]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [59]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [61]:
accuracy_score(ytrue, yhat)

1.0

# Real- Time

In [87]:
import cv2
import numpy as np
import mediapipe as mp

sequence = []
sentence = []
threshold = 0.4
collecting = False  # Flag to indicate collection state
frames_collected = 0

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)

        # Display instruction message
        if collecting:
            cv2.putText(image, "Perform the sign", (image.shape[1]//4, image.shape[0]//2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            frames_collected += 1
            
            if frames_collected == 30:
                # Display message when collection is over
                cv2.putText(image, "Frame collection complete", (image.shape[1]//4, image.shape[0]//2 + 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # Perform prediction
                input_data = np.expand_dims(sequence, axis=0).astype(np.float32)
                # input_data = scaler.transform(input_data.reshape(-1, 1662)).reshape(input_data.shape)
                print("Input data: ", input_data)
                res = model.predict(input_data)[0]
                
                sequence = []  # Reset sequence
                collecting = False  # Stop collection
                frames_collected = 0  # Reset frame counter
                print(res)
                
                # Confidence threshold check
                if float(res[np.argmax(res)]) > threshold:
                    if len(sentence) == 0 or actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                    
                if len(sentence) > 5:
                    sentence = sentence[-5:]  # Keep last 5 recognized words
        
        # Display recognized sentence
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Sign Language Recognition', image)
        
        # Key event handling
        key = cv2.waitKey(10) & 0xFF
        if key == ord('s') and not collecting:  # Start collection when 'S' is pressed
            collecting = True
            sequence = []  # Reset sequence
            frames_collected = 0  # Reset frame counter
            print("Collecting frames...")
        elif key == ord('q'):  # Quit program
            break

    cap.release()
    cv2.destroyAllWindows()


Input data:  [[[ 0.5036159   0.5118643  -0.93425447 ...  0.          0.
    0.        ]
  [ 0.5037027   0.5118582  -0.84539825 ...  0.          0.
    0.        ]
  [ 0.5037249   0.5116688  -0.8467848  ...  0.          0.
    0.        ]
  ...
  [ 0.49404234  0.50374454 -0.83344156 ...  0.          0.
    0.        ]
  [ 0.4951076   0.50290924 -0.81817234 ...  0.          0.
    0.        ]
  [ 0.49618864  0.50280714 -0.81700975 ...  0.          0.
    0.        ]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[0.8598579  0.01274831 0.1273938 ]
Input data:  [[[ 0.50133264  0.5134061  -0.9088832  ...  0.          0.
    0.        ]
  [ 0.50130945  0.51376283 -0.9016892  ...  0.          0.
    0.        ]
  [ 0.5012375   0.5137193  -0.8970129  ...  0.          0.
    0.        ]
  ...
  [ 0.488084    0.4817449  -0.71980023 ...  0.4432218   0.6174507
   -0.0473641 ]
  [ 0.48799333  0.48259696 -0.73992884 ...  0.41564783  0.63891345
   -0.05264014]
  [ 0.48552012  0.48272583 -0.76617146 ...  0.4